<a href="https://colab.research.google.com/github/jevabrir/ProyFin_Estad-stica/blob/main/Proy_PrecioCasas_UVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: f'{x:.3f}')


# Leer el archivo Excel
df = pd.read_excel('Datos_Kaggle_Ejem.xlsx')

# Ver las primeras 5 líneas del df
df.head(5)

In [ ]:
# Indica el tamaño del df (renglones, columnas)
df.shape

In [ ]:
df.columns

In [ ]:
df_numeric = df.select_dtypes(include=['number'])

# Convertir todas las columnas a valores numéricos (forzando errores a NaN si no se puede convertir)
df_numeric = df_numeric.apply(pd.to_numeric, errors='coerce')

# Imputar valores faltantes con la media de cada columna
df_imputado = df_numeric.fillna(df_numeric.mean())

# Calcular la matriz de correlación
corr_matrix = df_imputado.corr()

# Ordenar la matriz de correlación por la correlación absoluta con la variable objetivo (por ejemplo, 'SalePrice')
corr_sorted = corr_matrix['SalePrice'].sort_values(ascending=False)

# Seleccionar las variables más correlacionadas (puedes ajustar el número de variables que deseas ver)
top_corr = corr_sorted.head(10)  # Top 10 variables correlacionadas

# Crear la gráfica de correlación
plt.figure(figsize=(7, 6))
sns.heatmap(df_imputado[top_corr.index].corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Mapa de calor de la correlación de las principales variables')
plt.show()

In [ ]:
# Son las variables de interés

#'FullBath'    = Número de baños completos
# 1stFlrSF	   = Superficie del primer piso pies cuadrados
# 2ndFlrSF	   = Superficie del segundo piso
# BedroomAbvGr = Número de Dormitorios sobre rasante
# AntiqueYear  = Antiguedad casa a partir del año 2025
# SalePrice	   = Precio de venta en dólares

# Calcular la columna TotSF como la suma de 1stFlrSF + 1stFlrSF
df_imputado['TotSF'] = df_imputado['1stFlrSF'] + df_imputado['2ndFlrSF']


# Seleccionar columnas de interés
VariablesEstudio = ['SalePrice', 'TotSF', 'BedroomAbvGr', 'AntiqueYear']

# Generar df con variables de estudio
df_estudio = df_imputado[VariablesEstudio]

# Ver las primeras 5 líneas del df
df_estudio.head(5)

In [ ]:
df_estudio.info()

In [ ]:
# calcula los principales estadísticos del df
df_estudio.describe().T

In [ ]:
# Crear histogramas para todas las columnas numéricas
df_estudio.hist(bins=20, figsize=(10, 7), grid=True, edgecolor='black')

plt.suptitle('Histogramas de las Variables', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Datos de entrada

X1 = df_estudio['TotSF']	         # Tamaño en pies²
X2 = df_estudio['BedroomAbvGr']    # Número de habitaciones
X3 =df_estudio['AntiqueYear']      # Edad Casa (años)
Y= df_estudio['SalePrice']         # Precio casa en (dólares)


# Construcción de la matriz X con una columna de unos para la intersección
X_matrix = np.column_stack((np.ones(len(X1)), X1, X2, X3))

# Cálculo de los coeficientes usando la pseudoinversa
beta = np.linalg.pinv(X_matrix.T @ X_matrix) @ X_matrix.T @ Y

# Separación de coeficientes
b, a1, a2, a3 = beta

# Calcular valores predichos
Y_pred = X_matrix @ beta

# Calcular R^2
SS_res = np.sum((Y - Y_pred) ** 2)       # Suma de residuos al cuadrado
SS_tot = np.sum((Y - np.mean(Y)) ** 2)   # Suma total de cuadrados
R2 = 1 - (SS_res / SS_tot)

# Mostrar ecuación y R^2
print(f"Ecuación de regresión:\n")
print(f"Y = {a1:.3f} X1 + {a2:.3f} X2 + {a3:.3f} X3 + {b:.3f}\n")
print(f"Coeficiente de determinación (R²): {R2:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Crear figura con tamaño adecuado para 3 subgráficas
plt.figure(figsize=(18, 5))

# -------- Gráfica de Y vs X1 (Tamaño de la casa) --------
plt.subplot(1, 3, 1)
plt.scatter(X1, Y, color='blue', label='Datos reales')
plt.plot(X1, a1 * X1 + a2 * X2.mean() + a3 * X3.mean() + b, color='red', label='Regresión')
plt.xlabel("Tamaño de la casa (pies²)")
plt.ylabel("Precio (dólares)")
plt.title("Precio vs. Tamaño de la casa")
plt.legend()
plt.grid(True)

# -------- Gráfica de Y vs X2 (Número de habitaciones) --------
plt.subplot(1, 3, 2)
plt.scatter(X2, Y, color='green', label='Datos reales')
plt.plot(X2, a1 * X1.mean() + a2 * X2 + a3 * X3.mean() + b, color='red', label='Regresión')
plt.xlabel("Número de habitaciones")
plt.ylabel("Precio (dólares)")
plt.title("Precio vs. Número de habitaciones")
plt.legend()
plt.grid(True)

# -------- Gráfica de Y vs X3 (Edad de la casa, por ejemplo) --------
plt.subplot(1, 3, 3)
plt.scatter(X3, Y, color='orange', label='Datos reales')
plt.plot(X3, a1 * X1.mean() + a2 * X2.mean() + a3 * X3 + b, color='red', label='Regresión')
plt.xlabel("Edad de la casa (años)")
plt.ylabel("Precio (dólares)")
plt.title("Precio vs. Edad de la casa")
plt.legend()
plt.grid(True)

# Ajustar el layout y mostrar
plt.tight_layout()
plt.show()